In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import GaussianNB

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pickle
import random
from sklearn.metrics import accuracy_score,f1_score,classification_report
from sklearn.neighbors import KNeighborsClassifier

from tqdm import tqdm
import sys
import os
import warnings
if not sys.warnoptions:
	warnings.simplefilter("ignore")							#1
	os.environ["PYTHONWARNINGS"] = "ignore"					#2

	warnings.filterwarnings('ignore', 'Solver terminated early.*')		#3
	# warnings.filterwarnings('ignore', category="ConvergenceWarning")	#4
from datetime import datetime

In [3]:
m = datetime.now().month
d = datetime.now().day

In [3]:
# 데이터 읽기
train = pd.read_csv('./2023_ML_프로젝트_데이터/Train.csv')
test = pd.read_csv('./2023_ML_프로젝트_데이터/Test.csv')
# index drop
train = train.drop('Index',axis=1)
test = test.drop('Index',axis=1)
# train용 데이터의 독립변수와 종속변수의 분리
X = train.loc[:,:'X999']
y = train.iloc[:,-1]

In [82]:
# score 를 보기 위해 split, 제출용은 Full Train으로 제작할 것
def test_smote_model():
    # score 를 보기 위해 split, 제출용은 Full Train으로 제작할 것
    m = datetime.now().month
    d = datetime.now().day

    i = random.randint(1,1000)
    j = random.randint(1,1000)
    k = random.randint(1,1000)
    l = random.randint(1,1000)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=j)
    smote = SMOTE(random_state=i)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # 앙상블 방식
    classifiers = [
        RandomForestClassifier(n_estimators=30,random_state=k),
        LogisticRegression(max_iter=20),
        GradientBoostingClassifier(n_estimators=30),
        DecisionTreeClassifier(max_depth=20),
        GaussianNB(),
        # SVC(kernel="rbf") # 성능이 별로(param 세부조절을 안한 탓)
        # AdaBoostClassifier(n_estimators = 30),
        XGBClassifier(objective='binary:logistic'),
        MLPClassifier(random_state=l)
    ]
    result = random.sample(classifiers,3)

    a,b,c = result[0],result[1],result[2]
    
    weights1,weights2,weights3 = random.randint(1,6),random.randint(1,6),random.randint(1,6)
    model = VotingClassifier(estimators=[('1',a),('2',b),('3',c)],voting = 'soft',weights=[weights1,weights2,weights3])
    ensemble_train = model.fit(X_train_resampled,y_train_resampled)

    ensemble_pred = model.predict(X_test)
    score = f1_score(y_test,ensemble_pred,pos_label='positive')
    if round(score,2) >= 0.52:
        with open('model/smote_test_{}_{}월_{}일_({}_{})({},{},{}).pkl'.format(round(score,4),m,d,i,j,a,b,c),'wb') as f:
            pickle.dump(model,f)
    return round(score,4)


In [83]:
for i in tqdm(range(5)):
    a = test_smote_model()
    print(a)
# 실제로는 배치파일을 돌려서 200번~1000번 정도 진행함

 20%|██        | 1/5 [00:48<03:13, 48.45s/it]

0.262


 40%|████      | 2/5 [00:59<01:20, 26.74s/it]

0.341


 60%|██████    | 3/5 [01:44<01:09, 34.84s/it]

0.2656


 80%|████████  | 4/5 [02:42<00:44, 44.16s/it]

0.4216


100%|██████████| 5/5 [02:54<00:00, 34.89s/it]

0.2812


In [ ]:
ensemble_pred_for_csv = model.predict(test.loc[:,:'X999'])

print(f'Type : {type(ensemble_pred_for_csv)} | Length : {len(ensemble_pred_for_csv)}')

import numpy as np

np.savetxt('./버킷햇_0609.csv', ensemble_pred_for_csv, delimiter=',',fmt='%s')

In [14]:
with open('C:/Users/alseh/Desktop/jeonmincheol/2023/2023-1/binary_classification/model/smote_test_0.5.pkl','rb') as f:
    model = pickle.load(f)

In [15]:
model

VotingClassifier(estimators=[('lr', RandomForestClassifier(n_estimators=30)),
                             ('rf', LogisticRegression(max_iter=20)),
                             ('gnb', KNeighborsClassifier())],
                 voting='soft', weights=[5, 1, 1])